<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> Sara Giammusso, Samuel Pierre
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/ex1'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288088709
Epoch:  02   =====> Loss= 0.732281670
Epoch:  03   =====> Loss= 0.600392412
Epoch:  04   =====> Loss= 0.536762394
Epoch:  05   =====> Loss= 0.497826366
Epoch:  06   =====> Loss= 0.470822294
Epoch:  07   =====> Loss= 0.451210828
Epoch:  08   =====> Loss= 0.435753634
Epoch:  09   =====> Loss= 0.423504825
Epoch:  10   =====> Loss= 0.413171643
Epoch:  11   =====> Loss= 0.404602712
Epoch:  12   =====> Loss= 0.396665183
Epoch:  13   =====> Loss= 0.390432234
Epoch:  14   =====> Loss= 0.384589661
Epoch:  15   =====> Loss= 0.379198474
Epoch:  16   =====> Loss= 0.374669660
Epoch:  17   =====> Loss= 0.370596089
Epoch:  18   =====> Loss= 0.366341053
Epoch:  19   =====> Loss= 0.363085027
Epoch:  20   =====> Loss= 0.359674313
Epoch:  21   =====> Loss= 0.356840514
Epoch:  22   =====> Loss= 0.353889638
Epoch:  23   =====> Loss= 0.351291800
Epoch:  24   =====> Loss= 0.348949190
Epoch:  25   =====> Loss= 0.346355069
Epoch:  26   =====> Loss= 0.344331550
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [3]:
# Helper functions  for weigths and bias initilization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride, padding_):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding=padding_)

In [4]:
def LeNet5_Model(data, transfer="ReLU", keep_prob=1.):    
    
    transferFuncs = {"sigmoid" : tf.sigmoid, "ReLU": tf.nn.relu}
        
    #first convolutional layer
    W_conv1 = weight_variable([5, 5, 1, 6]) ## [filter_width, filter_height, depth_image_in, depth_image_out]
    b_conv1 = bias_variable([6])
    h_conv1 = transferFuncs[transfer](conv2d(data, W_conv1, 1, 'SAME') + b_conv1)
    pool1 = tf.nn.pool(h_conv1, [2,2], "MAX", 'VALID', strides=[2,2])
    
    #second convolutional layer
    W_conv2 = weight_variable([5, 5, 6, 16])
    b_conv2 = bias_variable([16])
    h_conv2 = transferFuncs[transfer](conv2d(pool1, W_conv2, 1, 'VALID') + b_conv2)
    pool2 = tf.nn.pool(h_conv2, [2,2], "MAX", 'VALID', strides=[2,2])
    
    #first fully connected layer
    s = pool2.get_shape().as_list()
    flattened_length = s[1] * s[2] * s[3]
    pool2_flat = tf.reshape(pool2, [-1, flattened_length])
    W_fc1 = weight_variable([1*5*5*16, 120])
    b_fc1 = bias_variable([120])
    h_fc1 = transferFuncs[transfer](tf.matmul(pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #second fully connected layer
    W_fc2 = weight_variable([120, 84])
    b_fc2 = bias_variable([84])
    h_fc2 = transferFuncs[transfer](tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
    
    #third fully connected layer
    W_fc3 = weight_variable([84, 10])
    b_fc3 = bias_variable([10])
    h_fc3 = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)
    
    return h_fc3

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

In [5]:
# first conv
pconv1 = 5*5*1*6 # filter_height * filter_width * channels_in * num_feature_maps
# second conv
pconv2 = 5*5*6*16 # filter_height * filter_width * channels_in * num_feature_maps
# first fcl
pfcl1 = 5*5*16*120 # fcl_input_size * fcl_output_size
# second fcl
pfcl2 = 84*120 # fcl_input_size * fcl_output_size
# third fcl
pfcl3 = 84*10 # fcl_input_size * fcl_output_size
# all the biases
pbias = 6 + 16 + 120 + 84 + 10 

total = pbias + pfcl1 + pfcl2 + pfcl3 + pconv2 + pconv1
print('Total number of parameters: ', total)

Total number of parameters:  61706


<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [6]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128

logs_path = 'log_files/ex2/'

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [7]:
def evaluate(model, y):
    correct = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    return tf.reduce_mean(tf.cast(correct, tf.float32))

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [8]:
import time 
def train(learning_rate, training_epochs, batch_size, display_step = 1, \
          logs_path='log_files/', optFunction="SGD", verbose=True, transfer="ReLU", keep_probability= 1.0):
    
    optFunctions = {"SGD":tf.train.GradientDescentOptimizer, "Adam":tf.train.AdamOptimizer}
    
    # Erase previous graph
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
    keep_prob = tf.placeholder(tf.float32)

    # Construct model
    with tf.name_scope('Model'):
        pred = LeNet5_Model(x, transfer=transfer)

    # Define loss and optimizer
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))

    with tf.name_scope(optFunction):
        if transfer is "sigmoid":
            optimizer = optFunctions[optFunction](learning_rate).minimize(cost)
        else:
            opt = optFunctions[optFunction](learning_rate)
            gvs = opt.compute_gradients(cost)
            capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
            optimizer = opt.apply_gradients(capped_gvs)

    # Evaluate model
    with tf.name_scope('Accuracy'):
        accuracy = evaluate(pred, y)

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", accuracy)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    x_val, y_val = mnist.validation.images.reshape(-1, 28, 28, 1), mnist.validation.labels
    x_test, y_test = mnist.test.images.reshape(-1, 28, 28, 1), mnist.test.labels

    with tf.Session() as sess:
        
        sess.run(init)
        if verbose is True:
            print("Start Training!")
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        saver = tf.train.Saver()
        # Training cycle
        start_time = time.time()
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle = True)
                batch_xs = batch_xs.reshape(-1, 28, 28, 1)
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys, keep_prob: keep_probability})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            
            val_acc = accuracy.eval({x: x_val, y:y_val, keep_prob:keep_probability})
            
            
            if verbose is True and (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), \
                      "  =====> Loss=", "{:.9f}".format(avg_cost), \
                      " Validation accuracy=", val_acc)
            if val_acc >= 0.99:
                if verbose is True:
                    print("Validation Accuracy over 99.0%% reached after %d epochs" %(epoch+1))
        
        elapsed_time = time.time() - start_time 
        saver.save(sess, 'Models/model_' + str(learning_rate) + '_' + str(batch_size) + '_' + optFunction)
        if verbose is True:
            print("Training Finished!")
            # Test model
            # Calculate accuracy
            print("Test accuracy:", accuracy.eval({x: x_test, y:y_test, keep_prob: keep_probability}))
            print("Elapsed time: ", elapsed_time, "sec")
        
    return elapsed_time

In [9]:
elapsed_time_1 = train(learning_rate, training_epochs, batch_size,logs_path=logs_path)

Start Training!
Epoch:  01   =====> Loss= 2.307532670  Validation accuracy= 0.1632
Epoch:  02   =====> Loss= 2.271728622  Validation accuracy= 0.221
Epoch:  03   =====> Loss= 2.232151651  Validation accuracy= 0.3728
Epoch:  04   =====> Loss= 2.163331968  Validation accuracy= 0.446
Epoch:  05   =====> Loss= 2.019186282  Validation accuracy= 0.5414
Epoch:  06   =====> Loss= 1.697722448  Validation accuracy= 0.6544
Epoch:  07   =====> Loss= 1.206095699  Validation accuracy= 0.7522
Epoch:  08   =====> Loss= 0.826219373  Validation accuracy= 0.8086
Epoch:  09   =====> Loss= 0.634631187  Validation accuracy= 0.8436
Epoch:  10   =====> Loss= 0.520621628  Validation accuracy= 0.8676
Epoch:  11   =====> Loss= 0.456320666  Validation accuracy= 0.8874
Epoch:  12   =====> Loss= 0.403626301  Validation accuracy= 0.8994
Epoch:  13   =====> Loss= 0.367017818  Validation accuracy= 0.9062
Epoch:  14   =====> Loss= 0.340801885  Validation accuracy= 0.9136
Epoch:  15   =====> Loss= 0.316848219  Validatio

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.


<img src="./MNIST_figures/loss1.png" style="width:80%">
  
<img src="./MNIST_figures/accuracy1.png" style="width:80%">
<center><span>Figure 2: Loss and Accuracy for LeNet5 with SGD</span></center>

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:

| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |       95.41%       |       98.24%        |       
| Training Time        |       11 min       |       11 min        |  

- Which optimizer gives the best accuracy on test data?

<div class="alert alert-success">
<strong>Answer 2.2.1 </strong><br>
The optimizer that gives the best accuracy on test data is AdamOptimizer. Indeed, with AdamOptimizer we reach 99% in just 8 epochs, while with SGD we never reach 99%, neither in 40 epochs. <br>
</div>

In [10]:
tf.reset_default_graph()
logs_path = 'log_files/ex3/'
elapsed_time_2 = train(learning_rate, training_epochs, batch_size, optFunction="Adam",logs_path=logs_path)

Start Training!
Epoch:  01   =====> Loss= 0.329192257  Validation accuracy= 0.9692
Epoch:  02   =====> Loss= 0.086178050  Validation accuracy= 0.9798
Epoch:  03   =====> Loss= 0.058729552  Validation accuracy= 0.986
Epoch:  04   =====> Loss= 0.045659533  Validation accuracy= 0.9866
Epoch:  05   =====> Loss= 0.039085610  Validation accuracy= 0.9876
Epoch:  06   =====> Loss= 0.030334682  Validation accuracy= 0.9896
Epoch:  07   =====> Loss= 0.026520149  Validation accuracy= 0.9876
Epoch:  08   =====> Loss= 0.024417924  Validation accuracy= 0.9904
Validation Accuracy over 99.0% reached after 8 epochs
Epoch:  09   =====> Loss= 0.019348163  Validation accuracy= 0.9888
Epoch:  10   =====> Loss= 0.017935880  Validation accuracy= 0.9886
Epoch:  11   =====> Loss= 0.015129979  Validation accuracy= 0.9904
Validation Accuracy over 99.0% reached after 11 epochs
Epoch:  12   =====> Loss= 0.013902304  Validation accuracy= 0.9908
Validation Accuracy over 99.0% reached after 12 epochs
Epoch:  13   ====


<img src="./MNIST_figures/loss2.png" style="width:80%">
  
<img src="./MNIST_figures/accuracy2.png" style="width:80%">
<center><span>Figure 3: Loss and Accuracy for LeNet5 with Adam</span></center>

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

In [11]:
tf.reset_default_graph()
logs_path = 'log_files/ex4/'
elapsed_time_3 = train(learning_rate, training_epochs, batch_size, optFunction="Adam", logs_path=logs_path, keep_probability= 0.75)

Start Training!
Epoch:  01   =====> Loss= 0.369031923  Validation accuracy= 0.9708
Epoch:  02   =====> Loss= 0.093144498  Validation accuracy= 0.9816
Epoch:  03   =====> Loss= 0.067406275  Validation accuracy= 0.9832
Epoch:  04   =====> Loss= 0.050735684  Validation accuracy= 0.9864
Epoch:  05   =====> Loss= 0.042091775  Validation accuracy= 0.9866
Epoch:  06   =====> Loss= 0.035156979  Validation accuracy= 0.986
Epoch:  07   =====> Loss= 0.030891549  Validation accuracy= 0.9886
Epoch:  08   =====> Loss= 0.027827556  Validation accuracy= 0.99
Validation Accuracy over 99.0% reached after 8 epochs
Epoch:  09   =====> Loss= 0.021516716  Validation accuracy= 0.9908
Validation Accuracy over 99.0% reached after 9 epochs
Epoch:  10   =====> Loss= 0.020065733  Validation accuracy= 0.9888
Epoch:  11   =====> Loss= 0.017868793  Validation accuracy= 0.9862
Epoch:  12   =====> Loss= 0.013813605  Validation accuracy= 0.9874
Epoch:  13   =====> Loss= 0.012400943  Validation accuracy= 0.9888
Epoch:  

<img  src="./MNIST_figures/loss3.png" style="width:80%">
<img src="./MNIST_figures/accuracy3.png" style="width:80%">
<center><span>Figure 3: Loss and Accuracy for LeNet5 with Adam and Dropout</span></center>

<div class="alert alert-success">
<strong>Answer 2.2.2 </strong><br>
Accuracy achieved on testing data: 99.24%.<br>
<p>We reach a better accuracy than the previous experiment!</p>
<p>Dropout refers to ignoring units (i.e. neurons) during the training phase of certain set of neurons which is chosen at random. By “ignoring”, we mean these units are not considered during a particular forward or backward pass.</p>
<p>Dropout is a good technique to prevent overfitting. Indeed, a fully connected layer occupies most of the parameters, and hence, neurons develop co-dependency amongst each other during training which curbs the individual power of each neuron leading to over-fitting of training data.</p>
</div>

## Final comparison

<img  src="./MNIST_figures/loss_ensemble.png" style="width:80%">
<img src="./MNIST_figures/acc_ensemble.png" style="width:80%">
<center><span>Figure 4: Loss and Accuracy final comparison</span></center>

<div class="alert alert-success">
<strong>Final comparison</strong>
<p>Figure 4 shows a comparison between accuracy and loss of the three models.<br>
The blue curve represents the model built using SGD, the red line the model built using AdamOptimizer and the turquoise curve the model built using AdamOptimizer and the Dropout.</p>

<p>SGD loss decreases more much slower than Adam loss. At the same time Adam accuracy is already very high at the very beginning. We don't notice particular differences between Adam model with and without dropout in these plots, even if the final test accuracy is 0.9924 using dropout and 0.9894 without it.</p>

<p>We can conclude that Adam is a better optimizer than SGD.</p>

</div>